# Cross Validation

As discussed in lectures, cross validation is a technique that allows us to efficiently use our available data for both training and evaluation purposes. It does this by assigning each observation in the training data to one of $k$ folds, and then one-by-one removing a fold from the data, fitting a model on the remaining data, and then evaluating the resulting model on the removed fold. This results in $k$ measures of performance, and to obtain the cross validation performance, we can (for example) simply take the mean of these values.

The following examples start with a long version approach to cross validation (for reference only - DO NOT USE THIS APPROACH), and then demonstrate how to apply cross validation through the functionality provided by scikit-learn.

## Our Data
We start with the following set of data (pertaining to a set of fuel efficiency readings for some rather old cars from the 1970s - around the time of the [Oil Crisis](https://en.wikipedia.org/wiki/1970s_energy_crisis)) :

In [1]:
import numpy as np
import pandas as pd

auto_data = pd.get_dummies(pd.read_csv('auto.csv'))

X, y, = auto_data.drop(columns=['mpg']).to_numpy(), auto_data['mpg'].to_numpy()

## Estimating generalisation performance through cross validation
We can use cross validation to obtain an estimate for the generalisation performance we can expect for a given model (which we can then use to honestly compare the performance of a set of candidate models). A hand-crafted way of doing this is as follows: (note: DO NOT USE THIS APPROACH!)

In [2]:
from sklearn.linear_model import LinearRegression

rng = np.random.default_rng(1234)

## set up the fold for each training instance
n_folds = 10
fold = np.array([ i % n_folds for i in range(len(X)) ])
rng.shuffle(fold)

## now, iterate over each fold
perf = []
for i in range(n_folds):
    ## split the data into training and testing sets according to the current fold
    X_train, X_test, y_train, y_test = X[np.where(fold != i)], X[np.where(fold == i)], y[np.where(fold != i)], y[np.where(fold == i)]
    ## fit a model on the current training set
    mdl = LinearRegression().fit(X_train, y_train)
    ## evaluate the performance of the model on the current test set, and append this to the list of performance results over all folds
    perf.append(mdl.score(X_test, y_test))

## use the perf results as an estimate of generalisation performance
print("Cross validation R-Squared performance of linear regression is {}".format(np.round(np.mean(perf), 3)))

Cross validation R-Squared performance of linear regression is 0.814


Cross validation is a ***very*** common activity. Therefore, scikit-learn provides an easy-to-use framework to perform cross validation. The above sequence can be replaced with the following equivalent code:

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression

kf = KFold(n_splits=10, shuffle=True, random_state=1234)
perf = cross_val_score(LinearRegression(), X, y, cv=kf)
print("Cross validation R-Squared performance of linear regression is {}".format(np.round(np.mean(perf), 3)))

Note that the score returned may not be _exactly_ the same (it is an estimate, after all) as the allocation of instances to folds has a random element to it. To reduce this effect, and therefore produce more stable estimates, you may wish to use the _repeated cross validation_ approach, which simply repeats the $k$-fold cross validation process a given number of times. For example, the above code can be slightly modified to perform 3 rounds of 10-fold cross validation as follows:

In [ ]:
from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.linear_model import LinearRegression

kf = RepeatedKFold(n_splits=10, n_repeats=3) ## note that this is the ONLY line of code that has changed!
perf = cross_val_score(LinearRegression(), X, y, cv=kf)
print("Cross validation R-Squared performance of linear regression is {}".format(np.round(np.mean(perf), 3)))

Looks like $R^{2}$ values of around 0.8 are typical for linear regression on this data.

Now that we have a simple method of obtaining cross validation estimates of performance for a learning algorithm, we can now use it to compare (and therfore _select_ from) a range of models. For example, we could compare linear regression to k-Nearest Neighbours (both with and without standardisation) as follows:

In [ ]:
from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor

## to incorporate feature standardisation into kNN, we need to create a pipeline that first performs standardisation, and then builds the model
zknn_pipe = Pipeline([
    ( 'scaler', StandardScaler()),
    ( 'model', KNeighborsRegressor(n_neighbors=3))
])

kf = RepeatedKFold(n_splits=10, n_repeats=3) ## note that this is the ONLY line of code that has changed!
lm = cross_val_score(LinearRegression(), X, y, cv=kf)
knn = cross_val_score(KNeighborsRegressor(n_neighbors=3), X, y, cv=kf)
zknn = cross_val_score(zknn_pipe, X, y, cv=kf)

print("Cross validation R-Squared performance of linear regression is {}".format(np.round(np.mean(lm), 3)))
print("Cross validation R-Squared performance of 3-neighbour kNN is {}".format(np.round(np.mean(knn), 3)))
print("Cross validation R-Squared performance of 3-neighbour standardised kNN is {}".format(np.round(np.mean(zknn), 3)))

Looks like standardised 3-neighbour kNN works well on this data (although it doesn't provide us any insights into the problem itself - but this is not what we're concerned with right now!). If desired, we can use the raw performance results rather than the aggregated values. For example, we could produce a boxplot of the results:

In [ ]:
import pandas as pd
import seaborn as sns

plot_data = pd.DataFrame({
    'Linear Regression' : lm,
    '3-Neighbour kNN' : knn,
    '3-Neighbour Standardised kNN' : zknn
})


ax = sns.boxplot(data=plot_data);
ax.figure.set_size_inches(10, 6)

## Hyperparameter optimisation via cross validation

As mentioned in the lecture, different combinations of hyperparameter settings produce different models, so we can treat the search for good hyperparameter settings as a model selection problem. The general idea is as follows:
1. Generate a list of possible hyperparameter settings
2. For each set of hyperparameter values in the list, compute the cross validation performance of the resulting model
3. Pick the set of hyperparameter values that produced the model with the highest cross validation performance
4. Build a model on the entire data set using the selected hyperparameter settings.

Lets look at the manual process that would be used for selecting the neighbourhood size for kNN: (again, this is for demonstration only - DO NOT USE THIS APPROACH)

In [ ]:
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor

rng = np.random.default_rng(1234)

## set up the hyperparameters that we'd like to explore - if we explore a lot of different 
## values for a large combination of hyperparameters, then this list could be VERY large
candidate_neighbourhoods = [ 1, 3, 5, 7, 9, 11, 31, 51, 71, 91, 101, 201 ]

## set up the fold for each training instance
n_folds = 10
fold = np.array([ i % n_folds for i in range(len(X)) ])
rng.shuffle(fold)

## now for each item in the hyperparameter list, compute the CV performance
scaler = StandardScaler()
neighbourhood_perf = []
for i, k in enumerate(candidate_neighbourhoods):    
    ## now, iterate over each fold
    perf = []
    for i in range(n_folds):
        ## split the data into training and testing sets according to the current fold
        X_train, X_test, y_train, y_test = X[np.where(fold != i)], X[np.where(fold == i)], y[np.where(fold != i)], y[np.where(fold == i)]
        Z_train, Z_test = scaler.fit_transform(X_train), scaler.transform(X_test)
        
        ## fit a model on the current training set
        mdl = KNeighborsRegressor(n_neighbors=k).fit(Z_train, y_train)
        ## evaluate the performance of the model on the current test set, and append this to the list of performance results over all folds
        perf.append(mdl.score(Z_test, y_test))
    neighbourhood_perf.append(np.mean(perf))

## extract the stats of the best hyperparameters
best_neighbourhood = np.argmax(neighbourhood_perf)
best_k = candidate_neighbourhoods[best_neighbourhood]
best_perf = neighbourhood_perf[best_neighbourhood]

## And now report
print("Cross validation suggests that a neighbourhood size of {} yields a model with performance of {}".format(best_k, np.round(best_perf, 3)))

Of course, we can plot this result to see the trend:

In [ ]:
import pandas as pd
import seaborn as sns

plot_data = pd.DataFrame({
    'k' : candidate_neighbourhoods,
    'CV R-Squared' : neighbourhood_perf
})

ax = sns.lineplot(data=plot_data, x='k', y='CV R-Squared')
sns.scatterplot(data=plot_data.iloc[[best_neighbourhood]], x='k', y='CV R-Squared', color='#ce2227');
ax.set_xlabel('k')
ax.set_ylabel('CV $R^{2}$')
ax.set_xscale('log');

From this plot, we can see that the performance of kNN is maximised at a neighbourhood size of 3, is roughly even between 3-7, and quickly degrades after a neighbourhood size $k$ greater than 7. Because their scores are so similar, it is likely than any neighbourhood size between 3 and 9 (inclusive) would be selected (due to the small fluctations we might expecte through cross validation).

As with evaluating a single model performance, scikit-learn provides a convenience framework from which to evaluate potential hyperparameter settings and fit the "optimal" model to data:

In [ ]:
import numpy as np

from sklearn.model_selection import RepeatedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor

kf = RepeatedKFold(n_splits=10, n_repeats=3) ## note that this is the ONLY line of code that has changed!

## to incorporate feature standardisation into kNN, we need to create a pipeline that first performs standardisation, and then builds the model
zknn_pipe = Pipeline([
    ( 'scaler', StandardScaler()),
    ( 'model', KNeighborsRegressor(n_neighbors=3))
])

## set up the hyperparameters that we'd like to explore - if we explore a lot of different 
## values for a large combination of hyperparameters, then this list could be VERY large
##
## the framework that we'll be using needs this to be set up as a dictionary with one
## entry for each hyperparameter - the framework will work out all the combinations
## for you
candidate_neighbourhoods = [ 1, 3, 5, 7, 9, 11, 31, 51, 71, 91, 101, 201 ]
param_grid = {
    'model__n_neighbors' : candidate_neighbourhoods
}

## now, construct the object that will search through the hyperparameters and fit the best model for you
cv = GridSearchCV(zknn_pipe, param_grid=param_grid, cv=kf)
cv.fit(X, y)

## extract the stats of the best hyperparameters
best_k = cv.best_params_
best_perf = cv.best_score_

## And now report
print("Cross validation suggests that a neighbourhood size of {} yields a model with performance of {}".format(best_k, np.round(best_perf, 3)))

The full details of the hyperparameter search are stored in the <code>cv_results_</code> property of the GridSearchCV object once the model has been fit:

In [ ]:
import pandas as pd

cv_data = pd.DataFrame(cv.cv_results_)
cv_data

And, as before, we can use this information to plot the results of our search to observe any useful trends:

In [ ]:
import seaborn as sns

ax = sns.lineplot(data=cv_data, x='param_model__n_neighbors', y='mean_test_score')
sns.scatterplot(data=cv_data.iloc[[cv.best_index_]], x='param_model__n_neighbors', y='mean_test_score', color='#ce2227');
ax.set_xlabel('k')
ax.set_ylabel('CV $R^{2}$')
ax.set_xscale('log');